In [1]:
from time import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown

In [2]:
model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

2024-05-16 09:26:22.114309: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 09:26:22.114424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 09:26:22.403218: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipeline.model.config.eos_token_id
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime

In [4]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [5]:
import json
# Specify the path to the JSON file
file_path = "/kaggle/input/segresnet-result-json/1_ms_lesion_areas_.json"

# Load the JSON file
with open(file_path, "r") as file:
    data = json.load(file)

# Now you can work with the loaded JSON data
print(data)

SegResNetResults = "/kaggle/input/segresnet-result-json/SegResNet_results.json"
with open(SegResNetResults, "r") as file:
    res = json.load(file)
print(res)

{'Frontal-to-Occipital (GapMap) left': {'Percentage of Lesion': 26.88, 'Percentage of Region': 6.04}, 'Frontal-to-Occipital (GapMap) right': {'Percentage of Lesion': 16.82, 'Percentage of Region': 4.08}, 'Area hOc1 (V1, 17, CalcS) left': {'Percentage of Lesion': 8.88, 'Percentage of Region': 7.31}, 'Area hOc6 (POS) left': {'Percentage of Lesion': 4.23, 'Percentage of Region': 51.55}, 'Frontal-to-Temporal-II (GapMap) left': {'Percentage of Lesion': 3.1, 'Percentage of Region': 2.86}}
{'nDSC': 71.75595231995298, 'nDSC std': 8.696038792793306, 'Lesion F1 score': 35.110906675494164, 'Lesion F1 score std': 14.263135368144157, 'nDSC R-AUC': 8.74497076414209, 'nDSC R-AUC std': 8.162803573241632}


In [6]:
PROMPT1 = """
Using the provided JSON data from a brain MRI study of the lesion segmentation computed by the AI model, please generate a comprehensive report detailing the segmentation results of MS lesions. Describe the lesions' presence in various regions according to the Julich-Brain Atlas noting the percentage  of the lesion area within each region and the impact on the region in order to enhance the explainability of the AI lesion segmentation model to support clinical decision-making. 
Here is the json:
"""
PROMPT1 = PROMPT1 + json.dumps(data) + json.dumps(res)+"\n."

In [ ]:
PROMPT2="""
Using the provided JSON data from a brain MRI study of the lesion segmentation computed by the AI model, please generate a comprehensive report detailing the segmentation results of MS lesions. 
Describe the lesions' presence in various regions according to the Julich-Brain Atlas noting the percentage of the lesion area within each region and the impact on the region in order to enhance the explainability of the AI lesion segmentation model to support clinical decision-making.

The report should be structured as follows:
1. **Summary of Lesion Segmentation**: Provide an overview of the lesions' distribution across various regions of the brain according to the Julich-Brain Atlas. Discuss the semantic segmentation of the lesions and their implications.
2. **Detailed Regional Impact**: For each region where the lesion is present, describe the percentage of the lesion within the region and the percentage of the region affected by the lesion. Discuss the known roles and functions of these regions in the brain and how the presence of the lesion might impact these functions.
3. **Semantic Segmentation**: Discuss the different aspects of the lesions identified by the semantic segmentation and their potential clinical implications.

Here is the json:
"""

PROMPT2 = PROMPT2 + json.dumps(data) + json.dumps(res)

In [7]:
system_message = """
You are an AI medical assistant designed to answer questions.
Please restrict your answer to the exact question asked.
"""
response = query_model(
    system_message,
    user_message=PROMPT1,
    temperature=0.1,
    max_length=1024)
display(Markdown(colorize_text(response)))
with open("/kaggle/working/llama3_8b_response_PROMPT1.txt", "w") as file:
    file.write(response)



**<font color='red'>Question:</font>** 
Using the provided JSON data from a brain MRI study of the lesion segmentation computed by the AI model, please generate a comprehensive report detailing the segmentation results of MS lesions. Describe the lesions' presence in various regions according to the Julich-Brain Atlas noting the percentage  of the lesion area within each region and the impact on the region in order to enhance the explainability of the AI lesion segmentation model to support clinical decision-making. 
Here is the json:
{"Frontal-to-Occipital (GapMap) left": {"Percentage of Lesion": 26.88, "Percentage of Region": 6.04}, "Frontal-to-Occipital (GapMap) right": {"Percentage of Lesion": 16.82, "Percentage of Region": 4.08}, "Area hOc1 (V1, 17, CalcS) left": {"Percentage of Lesion": 8.88, "Percentage of Region": 7.31}, "Area hOc6 (POS) left": {"Percentage of Lesion": 4.23, "Percentage of Region": 51.55}, "Frontal-to-Temporal-II (GapMap) left": {"Percentage of Lesion": 3.1, "Percentage of Region": 2.86}}{"nDSC": 71.75595231995298, "nDSC std": 8.696038792793306, "Lesion F1 score": 35.110906675494164, "Lesion F1 score std": 14.263135368144157, "nDSC R-AUC": 8.74497076414209, "nDSC R-AUC std": 8.162803573241632}
. 

**<font color='green'>Answer:</font>** Here is the comprehensive report detailing the segmentation results of MS lesions:

The AI model has segmented the MS lesions in the brain MRI study and provided the results in the following regions according to the Julich-Brain Atlas:

1. Frontal-to-Occipital (GapMap) left: The lesion occupies 26.88% of the lesion area and 6.04% of the total region. This suggests that the lesion has a moderate impact on this region.
2. Frontal-to-Occipital (GapMap) right: The lesion occupies 16.82% of the lesion area and 4.08% of the total region. This suggests that the lesion has a moderate impact on this region.
3. Area hOc1 (V1, 17, CalcS) left: The lesion occupies 8.88% of the lesion area and 7.31% of the total region. This suggests that the lesion has a significant impact on this region.
4. Area hOc6 (POS) left: The lesion occupies 4.23% of the lesion area and 51.55% of the total region. This suggests that the lesion has a significant impact on this region, potentially affecting the surrounding tissue.
5. Frontal-to-Temporal-II (GapMap) left: The lesion occupies 3.1% of the lesion area and 2.86% of the total region. This suggests that the lesion has a minor impact on this region.

The overall performance of the AI model is reflected in the metrics provided:

* nDSC (Normalized Distance to Similarity Curve): 71.75595231995298
* nDSC std: 8.696038792793306
* Lesion F1 score: 35.110906675494164
* Lesion F1 score std: 14.263135368144157
* nDSC R-AUC: 8.74497076414209
* nDSC R-AUC std: 8.162803573241632

These metrics indicate that the AI model has achieved a moderate level of accuracy in segmenting the MS lesions. However, the results should be interpreted in the context of the clinical presentation and other diagnostic findings to support informed decision-making. 

**<font color='magenta'>Total time:</font>** 36.95 sec.